In [1]:
import pandas as pd
import matplotlib as plt
from pathlib import Path

# Diretórios base
PROJECT_DIR = Path(".").resolve().parents[0] # sobe da pa sta notebooks para a raiz do projeto
DATA_RAW_DIR = PROJECT_DIR / "data" / "raw" / "olist"

PROJECT_DIR, DATA_RAW_DIR


(WindowsPath('C:/Enterprises/projects/ecommerce-demand-forecasting-3d-analytics'),
 WindowsPath('C:/Enterprises/projects/ecommerce-demand-forecasting-3d-analytics/data/raw/olist'))

In [4]:
orders = pd.read_csv(DATA_RAW_DIR / "olist_orders_dataset.csv")
order_items = pd.read_csv(DATA_RAW_DIR / "olist_order_items_dataset.csv")
customers = pd.read_csv(DATA_RAW_DIR / "olist_customers_dataset.csv")
products = pd.read_csv(DATA_RAW_DIR / "olist_products_dataset.csv")
payments = pd.read_csv(DATA_RAW_DIR / "olist_order_payments_dataset.csv")

orders.shape, order_items.shape, customers.shape, products.shape, payments.shape

((99441, 8), (112650, 7), (99441, 5), (32951, 9), (103886, 5))

### Visão geral das tabelas Olist

- `orders` → pedidos (nível do pedido).
- `order_items` → itens de cada pedido.
- `customers` → informações de clientes.
- `products` → catálogo de produtos.
- `payments` → pagamentos dos pedidos.

Tabela base para o modelo: **orders**.


In [5]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [7]:
orders.describe()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
count,99441,99441,99441,99441,99281,97658,96476,99441
unique,99441,99441,8,98875,90733,81018,95664,459
top,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-08-02 12:05:26,2018-02-27 04:31:10,2018-05-09 15:48:00,2018-05-08 19:36:48,2017-12-20 00:00:00
freq,1,1,96478,3,9,47,3,522


In [8]:
orders.isna().mean().sort_values(ascending=False)

order_delivered_customer_date    0.029817
order_delivered_carrier_date     0.017930
order_approved_at                0.001609
order_id                         0.000000
order_purchase_timestamp         0.000000
order_status                     0.000000
customer_id                      0.000000
order_estimated_delivery_date    0.000000
dtype: float64

In [9]:
orders.nunique().sort_values()

order_status                         8
order_estimated_delivery_date      459
order_delivered_carrier_date     81018
order_approved_at                90733
order_delivered_customer_date    95664
order_purchase_timestamp         98875
order_id                         99441
customer_id                      99441
dtype: int64

In [10]:
DOCS_DIR = PROJECT_DIR / "docs"
DOCS_DIR.mkdir(exist_ok=True)

DOCS_DIR

WindowsPath('C:/Enterprises/projects/ecommerce-demand-forecasting-3d-analytics/docs')

In [13]:
n_rows_orders = len(orders)
missing_pct_orders = orders.isna().mean().round(4)
n_rows_orders, missing_pct_orders

(99441,
 order_id                         0.0000
 customer_id                      0.0000
 order_status                     0.0000
 order_purchase_timestamp         0.0000
 order_approved_at                0.0016
 order_delivered_carrier_date     0.0179
 order_delivered_customer_date    0.0298
 order_estimated_delivery_date    0.0000
 dtype: float64)

In [14]:
data_dict_orders = pd.DataFrame([
    {
        "tabela": "orders",
        "coluna": "order_id",
        "pandas_dtype": str(orders["order_id"].dtype),
        "tipo_semantico": "id_pedido",
        "descricao": "Identificador único do pedido na base Olist.",
        "eh_chave": "primaria",
        "pct_missing": float(missing_pct_orders["order_id"]),
        "comentarios": "Usar apenas para joins e rastreio, não como feature."
    },
    {
        "tabela": "orders",
        "coluna": "customer_id",
        "pandas_dtype": str(orders["customer_id"].dtype),
        "tipo_semantico": "id_cliente",
        "descricao": "Identificador do cliente que fez o pedido.",
        "eh_chave": "estrangeira",
        "pct_missing": float(missing_pct_orders["customer_id"]),
        "comentarios": "Chave estrangeira para tabela de clientes. Não usar como feature (ID puro)."
    },
    {
        "tabela": "orders",
        "coluna": "order_status",
        "pandas_dtype": str(orders["order_status"].dtype),
        "tipo_semantico": "categoria_status_pedido",
        "descricao": "Status do pedido (delivered, shipped, cancelled, etc.).",
        "eh_chave": "nenhuma",
        "pct_missing": float(missing_pct_orders["order_status"]),
        "comentarios": "Boa para filtros e EDA. Em alguns cenários pode ser alvo."
    },
    {
        "tabela": "orders",
        "coluna": "order_purchase_timestamp",
        "pandas_dtype": str(orders["order_purchase_timestamp"].dtype),
        "tipo_semantico": "timestamp_compra",
        "descricao": "Data e hora em que o pedido foi feito pelo cliente.",
        "eh_chave": "nenhuma",
        "pct_missing": float(missing_pct_orders["order_purchase_timestamp"]),
        "comentarios": "Candidata a 'data de referência' para splits temporais. Converter para datetime depois."
    },
    {
        "tabela": "orders",
        "coluna": "order_approved_at",
        "pandas_dtype": str(orders["order_approved_at"].dtype),
        "tipo_semantico": "timestamp_aprovacao_pagamento",
        "descricao": "Data e hora em que o pagamento do pedido foi aprovado.",
        "eh_chave": "nenhuma",
        "pct_missing": float(missing_pct_orders["order_approved_at"]),
        "comentarios": "Acontece após a compra → informação de futuro; não usar como feature se o modelo decide na compra."
    },
    {
        "tabela": "orders",
        "coluna": "order_delivered_carrier_date",
        "pandas_dtype": str(orders["order_delivered_carrier_date"].dtype),
        "tipo_semantico": "timestamp_envio_transportadora",
        "descricao": "Data em que o pedido foi entregue à transportadora.",
        "eh_chave": "nenhuma",
        "pct_missing": float(missing_pct_orders["order_delivered_carrier_date"]),
        "comentarios": "Também após a compra. Boa para análise de logística, mas é futuro → não usar como feature."
    },
    {
        "tabela": "orders",
        "coluna": "order_delivered_customer_date",
        "pandas_dtype": str(orders["order_delivered_customer_date"].dtype),
        "tipo_semantico": "timestamp_entrega_cliente",
        "descricao": "Data em que o pedido foi entregue ao cliente.",
        "eh_chave": "nenhuma",
        "pct_missing": float(missing_pct_orders["order_delivered_customer_date"]),
        "comentarios": "Futuro claro. Candidata a alvo (prazo, atraso), nunca feature na compra."
    },
    {
        "tabela": "orders",
        "coluna": "order_estimated_delivery_date",
        "pandas_dtype": str(orders["order_estimated_delivery_date"].dtype),
        "tipo_semantico": "timestamp_estimativa_entrega",
        "descricao": "Data estimada de entrega informada ao cliente.",
        "eh_chave": "nenhuma",
        "pct_missing": float(missing_pct_orders["order_estimated_delivery_date"]),
        "comentarios": "Estimativa conhecida na compra. Pode ser feature se você aceitar usar essa informação do sistema."
    },
])

data_dict_orders


,tabela,coluna,pandas_dtype,tipo_semantico,descricao,eh_chave,pct_missing,comentarios
0,orders,order_id,object,id_pedido,Identificador único do pedido na base Olist.,primaria,0.0000,"Usar apenas para joins e rastreio, não como fe..."
1,orders,customer_id,object,id_cliente,Identificador do cliente que fez o pedido.,estrangeira,0.0000,Chave estrangeira para tabela de clientes. Não...
2,orders,order_status,object,categoria_status_pedido,"Status do pedido (delivered, shipped, cancelle...",nenhuma,0.0000,Boa para filtros e EDA. Em alguns cenários pod...
3,orders,order_purchase_timestamp,object,timestamp_compra,Data e hora em que o pedido foi feito pelo cli...,nenhuma,0.0000,Candidata a 'data de referência' para splits t...
4,orders,order_approved_at,object,timestamp_aprovacao_pagamento,Data e hora em que o pagamento do pedido foi a...,nenhuma,0.0016,Acontece após a compra → informação de futuro;...
5,orders,order_delivered_carrier_date,object,timestamp_envio_transportadora,Data em que o pedido foi entregue à transporta...,nenhuma,0.0179,Também após a compra. Boa para análise de logí...
6,orders,order_delivered_customer_date,object,timestamp_entrega_cliente,Data em que o pedido foi entregue ao cliente.,nenhuma,0.0298,"Futuro claro. Candidata a alvo (prazo, atraso)..."
7,orders,order_estimated_delivery_date,object,timestamp_estimativa_entrega,Data estimada de entrega informada ao cliente.,nenhuma,0.0000,Estimativa conhecida na compra. Pode ser featu...


In [15]:
data_dict_orders_path = DOCS_DIR / "data_dictionary_orders.csv"
data_dict_orders.to_csv(data_dict_orders_path, index=False)

data_dict_orders_path


WindowsPath('C:/Enterprises/projects/ecommerce-demand-forecasting-3d-analytics/docs/data_dictionary_orders.csv')

In [17]:
time_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]

orders[time_cols] = orders[time_cols].apply(pd.to_datetime)

orders[time_cols].dtypes

order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [18]:
tempo_md_path = DOCS_DIR / "tempo_orders.md"

tempo_md_content = """# Chaves de tempo – tabela orders (Olist)

## Coluna de referência (momento da decisão)

- `order_purchase_timestamp`
  - Momento em que o cliente conclui o pedido.
  - Vamos tratar como a **data de referência** do modelo (o "agora" na hora da previsão).
  - É com base nela que vamos:
    - Fazer splits temporais (treino = períodos mais antigos, teste = períodos mais recentes).
    - Decidir o que é passado/presente vs. futuro.

## Colunas que acontecem DEPOIS da compra (futuro → não usar como feature)

- `order_approved_at`
  - Papel: data/hora em que o pagamento é aprovado.
  - Acontece após a compra.
  - Informação de FUTURO em relação à `order_purchase_timestamp`.
  - Regra: **não usar como feature** em modelos que tomam decisão na hora da compra.

- `order_delivered_carrier_date`
  - Papel: data em que o pedido é entregue à transportadora.
  - Sempre depois da compra.
  - Informação de logística futura. Pode ser usada para análise de processo, mas:
    - **Não usar como feature** na previsão feita na compra.

- `order_delivered_customer_date`
  - Papel: data em que o pedido é entregue ao cliente.
  - Claramente bem depois da compra.
  - Natural candidata a **alvo** em problemas de prazo/atraso.
  - **Nunca** deve ser usada como feature em modelos que “decidem” na compra.

## Coluna de estimativa de entrega

- `order_estimated_delivery_date`
  - Papel: data estimada de entrega informada ao cliente.
  - Em princípio, conhecida pelo sistema na hora da compra.
  - Pode ser usada como feature se a regra de negócio permitir “usar o que o sistema já estimava”.
  - Se for usada como feature, documentar claramente:
    - O modelo está usando uma informação que também é derivada de dados históricos de logística.

## Resumo de regra anti–vazamento

- Tudo que acontece DEPOIS de `order_purchase_timestamp` é FUTURO para um modelo que decide na compra.
- Essas colunas de futuro (aprovado, enviado, entregue) **não podem entrar como features**.
- Elas podem:
  - ser alvo (ex.: prazo de entrega),
  - servir para análise de processo (EDA, dashboards),
  - mas não como entrada do modelo de previsão de demanda/prazo na data da compra.
"""

tempo_md_path.write_text(tempo_md_content, encoding="utf-8")
tempo_md_path


WindowsPath('C:/Enterprises/projects/ecommerce-demand-forecasting-3d-analytics/docs/tempo_orders.md')